In [1]:
import Preprocess as prep
from itertools import combinations
import concurrent.futures
import itertools

#food_db = prep.processed_food()
#a='p'
food_db = prep.general_food()
a='d'

selected_columns = ['카테고리','식품코드','식품명','영양성분함량기준량', '에너지(kcal)', '탄수화물(g)', '단백질(g)', '지방(g)', '나트륨(mg)', '당류(g)', '칼륨(mg)', '인(mg)']
food_db = food_db[selected_columns]
food_db = food_db.dropna(subset=['탄수화물(g)', '단백질(g)', '지방(g)', '나트륨(mg)', '당류(g)'])

def foods_list(category):
    food_list = food_db[food_db['카테고리'] == category][['영양성분함량기준량','식품코드', '식품명']].values.tolist()
    return food_list

def calculate_diet_nutrition(foods, selected_cols):
    diet_nutrition = food_db.loc[food_db['식품코드'].isin(food[1] for food in foods), selected_cols]
    nutritions = diet_nutrition.values.tolist() # .fillna(0).sum(axis=0)
    return nutritions

def process_diet(diet):
    nutrition = calculate_diet_nutrition(diet, selected_cols)
    return diet, nutrition

def generate_diet_combinations(category, num_items):
    food_list = foods_list(category)
    diet_combinations = list(combinations(food_list, num_items))
    return diet_combinations

selected_cols = food_db.columns[4:].values 

# 음식

## 간식

### 간식

In [3]:
resulting_diet, total_nutrition = [], []
snack_combinations = generate_diet_combinations('간식', 1)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, snack_combinations)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'processed_DB/snack/snack{a}.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")

### 음료

In [4]:
resulting_diet, total_nutrition = [], []
snack_combinations = generate_diet_combinations('음료', 1)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, snack_combinations)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'processed_DB/beverage/beverage{a}.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")

### 간식+음료 / 간식2개

In [3]:
# 간식1 + 음료1
resulting_diet, total_nutrition = [], []
snack_combination1=[]
for sn in foods_list('간식'):
    for b in foods_list('음료'):
        snack_diet1 = (sn,b)
        snack_combination1.append(snack_diet1)
chunks = [snack_combination1[i:i+20000] for i in range(0, len(snack_combination1), 20000)]

# 각 chunk에 대한 처리 및 저장
for n, chunk in enumerate(chunks):
    resulting_diet = []
    total_nutrition = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(process_diet, chunk)

    for diet, nutrition in results:
        resulting_diet.append(diet)
        total_nutrition.append(nutrition)

    with open(f'processed_DB/snack_combi/snack_combi{a}1.txt', 'w') as f:
        for diet, nutrition in zip(resulting_diet, total_nutrition):
            f.write(f"Diet: {diet}\n")
            f.write(f"{nutrition}\n")
            f.write("\n")

In [4]:
# 간식2
resulting_diet, total_nutrition = [], []
snack_combinations = generate_diet_combinations('간식', 2)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, snack_combinations)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'processed_DB/snack_combi/snack_combi{a}2.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")

## 식사

### 식사 조합 (diet)

In [2]:
print(len(foods_list('밥')))
print(len(foods_list('국&찌개')))
print(len(foods_list('주반찬')))
print(len(foods_list('부수반찬')))
print(len(foods_list('김치')))
print(len(foods_list('주식')))

1
23
55
80
10
52


In [3]:
# 밥1+주반찬1+부수반찬1+김치+국&찌개1
resulting_diet, total_nutrition = [], []
main_combination=[]
for r in foods_list('밥'):
    for sp in foods_list('국&찌개'):
        for sd1 in foods_list('주반찬'):
            for sd2 in foods_list('부수반찬'):    
                for sd3 in foods_list('김치'):
                    main_diet = (r, sd1, sd2, sd3, sp) # 
                    main_combination.append(main_diet)

print(len(main_combination))

chunks = [main_combination[i:i+20000] for i in range(0, len(main_combination), 20000)]

for n, chunk in enumerate(chunks):
    resulting_diet = []
    total_nutrition = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(process_diet, chunk)

    for diet, nutrition in results:
        resulting_diet.append(diet)
        total_nutrition.append(nutrition)

    with open(f'processed_DB/diet/diet{a}1_{n}.txt', 'w') as f:
        for diet, nutrition in zip(resulting_diet, total_nutrition):
            f.write(f"Diet: {diet}\n")
            f.write(f"{nutrition}\n")
            f.write("\n")

1012000


In [4]:
# 주식1+부수반찬2+김치+국&찌개1
side_dish_combinations2 = generate_diet_combinations('부수반찬', 2)
resulting_diet, total_nutrition = [], []
main_combination=[]
for r in foods_list('주식'):
    for sp in foods_list('국&찌개'):
        for (sd1,sd2) in side_dish_combinations2:
            for sd3 in foods_list('김치'):
                main_diet = (r, sd1, sd2, sd3, sp) # 
                main_combination.append(main_diet)

print(len(main_combination))

chunks = [main_combination[i:i+20000] for i in range(0, len(main_combination), 20000)]

for n, chunk in enumerate(chunks):
    resulting_diet = []
    total_nutrition = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(process_diet, chunk)

    for diet, nutrition in results:
        resulting_diet.append(diet)
        total_nutrition.append(nutrition)

    with open(f'processed_DB/diet/diet{a}2_{n}.txt', 'w') as f:
        for diet, nutrition in zip(resulting_diet, total_nutrition):
            f.write(f"Diet: {diet}\n")
            f.write(f"{nutrition}\n")
            f.write("\n")

37793600


### 한그릇 식사 (one)

In [ ]:
# 주식
resulting_diet, total_nutrition = [], []
main_combinations = generate_diet_combinations('주식', 1)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, main_combinations)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'processed_DB/one/one{a}1.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")

### 주반찬 (main)

In [2]:
# 주반찬
resulting_diet, total_nutrition = [], []
diet_combination=[]
for sd1 in food_db[food_db['카테고리'] == '주반찬'][['영양성분함량기준량','식품코드', '식품명']].values.tolist():
    r = list(itertools.chain(*food_db[food_db['식품명']=='차조밥'][['영양성분함량기준량','식품코드', '식품명']].values.tolist()))
    sd2 = list(itertools.chain(*food_db[food_db['식품명']=='겉절이_배추'][['영양성분함량기준량','식품코드', '식품명']].values.tolist()))
    diet = (sd1, r, sd2,)
    diet_combination.append(diet)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, diet_combination)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'processed_DB/main/main{a}.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")

# 가공식품 조합

## main

In [2]:
# 일품1
resulting_diet, total_nutrition = [], []
main_combinations = generate_diet_combinations('일품', 1)
with concurrent.futures.ThreadPoolExecutor() as executor:
    results = executor.map(process_diet, main_combinations)
for diet, nutrition in results:
    resulting_diet.append(diet)
    total_nutrition.append(nutrition)
with open(f'main_combination/main{a}1.txt', 'w') as f:
    for diet, nutrition in zip(resulting_diet, total_nutrition):
        f.write(f"Diet: {diet}\n")
        f.write(f"{nutrition}\n")
        f.write("\n")